## EDA Workflow

In [ ]:
# FAWN Weather Data - Exploratory Data Analysis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
from datetime import datetime
warnings.filterwarnings('ignore')

In [ ]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


print("STEP 1: LOAD & INITIAL RECONNAISSANCE")
# Load the data
df = pd.read_csv('c:/Users/USER/Downloads/FAWN_report.csv')

print("\n1.1 BASIC DATASET INFORMATION")
print(f"Dataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n1.2 COLUMN NAMES AND TYPES")
print(df.dtypes)

print("\n1.3 FIRST FEW ROWS")
print(df.head(10))

print("\n1.4 LAST FEW ROWS")
print(df.tail(10))

print("\n1.5 BASIC STATISTICS")
print(df.describe())

print("\n1.6 DATASET INFO")
df.info()

print("\n1.7 UNIQUE VALUES IN CATEGORICAL COLUMNS")
print(f"Number of unique FAWN Stations: {df['FAWN Station'].nunique()}")
print(f"Unique Stations: {df['FAWN Station'].unique()}")
print(f"\nNumber of unique Periods: {df['Period'].nunique()}")
print(f"Sample Periods: {df['Period'].unique()[:10]}")



In [ ]:
print("STEP 2: DATA QUALITY ASSESSMENT")

print("\n2.1 MISSING VALUES ANALYSIS")
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Missing_Percentage': missing_pct
}).sort_values('Missing_Count', ascending=False)

if missing_df['Missing_Count'].sum() > 0:
    print(missing_df[missing_df['Missing_Count'] > 0])
else:
    print("No missing values detected in the dataset!")
    print("\nAll columns have complete data:")
    print(missing_df.head())

# Visualize missing data
if missing_df['Missing_Count'].sum() > 0:
    plt.figure(figsize=(12, 6))
    missing_df[missing_df['Missing_Count'] > 0].plot(kind='bar', y='Missing_Percentage')
    plt.title('Missing Data Percentage by Column')
    plt.ylabel('Percentage Missing (%)')
    plt.xlabel('Columns')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print(" No missing values found in the dataset!")

print("\n2.2 DATA TYPES CHECK")
print("Expected types vs Actual types:")
for col in df.columns:
    print(f"{col}: {df[col].dtype}")

# Check if Period should be datetime
print(f"\nSample Period values: {df['Period'].head()}")

print("\n2.3 DUPLICATE ROWS")
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
if duplicates > 0:
    print(f"Percentage of duplicates: {(duplicates/len(df))*100:.2f}%")

print("\n2.4 OUTLIER DETECTION (IQR Method)")
numeric_cols = df.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    
    if len(outliers) > 0:
        print(f"\n{col}:")
        print(f"  Outliers: {len(outliers)} ({(len(outliers)/len(df))*100:.2f}%)")
        print(f"  Range: [{df[col].min():.2f}, {df[col].max():.2f}]")
        print(f"  Expected range (IQR): [{lower_bound:.2f}, {upper_bound:.2f}]")

print("\n2.5 ZERO/NEGATIVE VALUES CHECK")
for col in numeric_cols:
    zeros = (df[col] == 0).sum()
    negatives = (df[col] < 0).sum()
    if zeros > 0 or negatives > 0:
        print(f"{col}: {zeros} zeros, {negatives} negative values")

print("\n2.6 CONSISTENCY CHECKS")
# Check if min < avg < max for temperature
temp_issues = df[
    (df['2m T min (F)'] > df['2m T avg (F)']) | 
    (df['2m T avg (F)'] > df['2m T max (F)'])
]
print(f"Temperature consistency issues: {len(temp_issues)} rows")

# Check if soil temp min < avg < max
soil_issues = df[
    (df['Tsoil min(avg)-10cm  (F)'] > df['Tsoil avg-10cm  (F)']) | 
    (df['Tsoil avg-10cm  (F)'] > df['Tsoil max(avg)-10cm  (F)'])
]
print(f"Soil temperature consistency issues: {len(soil_issues)} rows")

# Check if wind min < avg < max
wind_issues = df[
    (df['10m Wind min (mph)'] > df['10m Wind avg (mph)']) | 
    (df['10m Wind avg (mph)'] > df['10m Wind max (mph)'])
]
print(f"Wind speed consistency issues: {len(wind_issues)} rows")

print("\n2.7 UNIQUE STATION ANALYSIS")
station_counts = df['FAWN Station'].value_counts()
print(f"Total stations: {len(station_counts)}")
print(f"\nObservations per station:")
print(station_counts.head(10))

print("\n2.8 DATA QUALITY SUMMARY")
print("Issues Found:")
print(f" Missing values: {missing_df['Missing_Count'].sum()} total")
print(f" Duplicates: {duplicates}")
print(f" Potential outliers detected in multiple variables")
print(f" Consistency issues in min/avg/max relationships")

In [ ]:
print("STEP 3: CLEANING DECISIONS AND IMPLEMENTATION")

print(f"\nOriginal dataset shape: {df.shape}")

# Create a copy for cleaning
df_clean = df.copy()

print("\n3.1 HANDLE DUPLICATES")
duplicates_before = df_clean.duplicated().sum()
df_clean = df_clean.drop_duplicates()
print(f"Duplicates removed: {duplicates_before}")
print(f"New shape: {df_clean.shape}")

print("\n3.2 FIX DATA TYPES")
# Convert Period to datetime
try:
    df_clean['Period'] = pd.to_datetime(df_clean['Period'])
    print(" Period converted to datetime")
except:
    print(" Could not convert Period to datetime - keeping as string")

# Ensure numeric columns are numeric
numeric_columns = [
    '2m T avg (F)', '2m T min (F)', '2m T max (F)',
    'Tsoil avg-10cm  (F)', 'Tsoil min(avg)-10cm  (F)', 'Tsoil max(avg)-10cm  (F)',
    '2m DewPt avg (F)', 'RelHum avg 2m  (pct)', '2m Rain tot (in)',
    '2m Rain max over 15min (in)', 'SolRad avg2m  (w/m^2)',
    '10m Wind avg (mph)', '10m Wind min (mph)', '10m Wind max (mph)',
    'WDir avg10m  (deg)', 'BP avg (mb)', 'N (# obs)', '2m WetBulb (F)'
]

for col in numeric_columns:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
print(f" Numeric columns verified")

print("\n3.3 HANDLE MISSING VALUES")
missing_before = df_clean.isnull().sum().sum()

# Decision rules for missing values:
# 1. If a row has > 50% missing values, remove it
threshold = 0.5 * len(df_clean.columns)
df_clean = df_clean.dropna(thresh=threshold)
print(f"Rows with >50% missing removed: {len(df) - len(df_clean)}")

# 2. For specific columns with few missing values, use forward fill or interpolation
for col in numeric_columns:
    missing_pct = df_clean[col].isnull().sum() / len(df_clean)
    if 0 < missing_pct < 0.05:  # Less than 5% missing
        df_clean[col] = df_clean[col].interpolate(method='linear')
        print(f" {col}: Interpolated {missing_pct*100:.2f}% missing values")

missing_after = df_clean.isnull().sum().sum()
print(f"\nMissing values: {missing_before} → {missing_after}")

print("\n3.4 HANDLE OUTLIERS")
print("Decision: Keep outliers but flag them for investigation")
print("Rationale: Weather data can have legitimate extreme values")

# Flag extreme outliers (beyond 3 standard deviations)
df_clean['has_outlier'] = False
for col in numeric_columns:
    if col in df_clean.columns:
        mean = df_clean[col].mean()
        std = df_clean[col].std()
        outliers = (df_clean[col] < mean - 3*std) | (df_clean[col] > mean + 3*std)
        df_clean.loc[outliers, 'has_outlier'] = True

outlier_count = df_clean['has_outlier'].sum()
print(f"Rows flagged with extreme outliers: {outlier_count} ({outlier_count/len(df_clean)*100:.2f}%)")

print("\n3.5 FIX CONSISTENCY ISSUES")
# Fix temperature min/max issues
temp_fixes = 0
mask = df_clean['2m T min (F)'] > df_clean['2m T max (F)']
if mask.sum() > 0:
    # Swap min and max if inverted
    df_clean.loc[mask, ['2m T min (F)', '2m T max (F)']] = df_clean.loc[mask, ['2m T max (F)', '2m T min (F)']].values
    temp_fixes = mask.sum()
print(f"Temperature min/max swapped: {temp_fixes} rows")

# Fix soil temperature issues
soil_fixes = 0
mask = df_clean['Tsoil min(avg)-10cm  (F)'] > df_clean['Tsoil max(avg)-10cm  (F)']
if mask.sum() > 0:
    df_clean.loc[mask, ['Tsoil min(avg)-10cm  (F)', 'Tsoil max(avg)-10cm  (F)']] = \
        df_clean.loc[mask, ['Tsoil max(avg)-10cm  (F)', 'Tsoil min(avg)-10cm  (F)']].values
    soil_fixes = mask.sum()
print(f"Soil temperature min/max swapped: {soil_fixes} rows")

# Fix wind speed issues
wind_fixes = 0
mask = df_clean['10m Wind min (mph)'] > df_clean['10m Wind max (mph)']
if mask.sum() > 0:
    df_clean.loc[mask, ['10m Wind min (mph)', '10m Wind max (mph)']] = \
        df_clean.loc[mask, ['10m Wind max (mph)', '10m Wind min (mph)']].values
    wind_fixes = mask.sum()
print(f"Wind speed min/max swapped: {wind_fixes} rows")

print("\n3.6 REMOVE INVALID RECORDS")
# Remove records with impossible values
initial_len = len(df_clean)

# Temperature should be within reasonable Florida range
df_clean = df_clean[
    (df_clean['2m T avg (F)'] >= -20) & 
    (df_clean['2m T avg (F)'] <= 120)
]

# Humidity should be 0-100%
df_clean = df_clean[
    (df_clean['RelHum avg 2m  (pct)'] >= 0) & 
    (df_clean['RelHum avg 2m  (pct)'] <= 100)
]

# Rain cannot be negative
df_clean = df_clean[df_clean['2m Rain tot (in)'] >= 0]

# Wind direction should be 0-360 degrees
df_clean = df_clean[
    (df_clean['WDir avg10m  (deg)'] >= 0) & 
    (df_clean['WDir avg10m  (deg)'] <= 360)
]

removed = initial_len - len(df_clean)
print(f"Invalid records removed: {removed}")

print("\n3.7 SAVE CLEANED DATA")
df_clean.to_csv('FAWN_report_cleaned.csv', index=False)
print(" Cleaned data saved to: FAWN_report_cleaned.csv")

print("\n3.8 CLEANING SUMMARY")
print(f"Original shape: {df.shape}")
print(f"Cleaned shape: {df_clean.shape}")
print(f"Rows removed: {len(df) - len(df_clean)} ({((len(df) - len(df_clean))/len(df))*100:.2f}%)")
print(f"Columns added: 1 (has_outlier flag)")

In [ ]:
print("STEP 4: STATISTICAL EXPLORATORY DATA ANALYSIS")

# UNIVARIATE ANALYSIS
print("\n4.1 UNIVARIATE ANALYSIS - DISTRIBUTIONS")

# Temperature distribution
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Weather Variables Distributions', fontsize=16)

# Temperature
axes[0, 0].hist(df['2m T avg (F)'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Air Temperature Distribution')
axes[0, 0].set_xlabel('Temperature (F)')
axes[0, 0].set_ylabel('Frequency')

# Humidity
axes[0, 1].hist(df['RelHum avg 2m  (pct)'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[0, 1].set_title('Relative Humidity Distribution')
axes[0, 1].set_xlabel('Humidity (%)')
axes[0, 1].set_ylabel('Frequency')

# Rainfall
axes[0, 2].hist(df['2m Rain tot (in)'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='blue')
axes[0, 2].set_title('Total Rainfall Distribution')
axes[0, 2].set_xlabel('Rainfall (inches)')
axes[0, 2].set_ylabel('Frequency')

# Wind Speed
axes[1, 0].hist(df['10m Wind avg (mph)'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 0].set_title('Wind Speed Distribution')
axes[1, 0].set_xlabel('Wind Speed (mph)')
axes[1, 0].set_ylabel('Frequency')

# Solar Radiation
axes[1, 1].hist(df['SolRad avg2m  (w/m^2)'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='yellow')
axes[1, 1].set_title('Solar Radiation Distribution')
axes[1, 1].set_xlabel('Solar Radiation (w/m²)')
axes[1, 1].set_ylabel('Frequency')

# Barometric Pressure
axes[1, 2].hist(df['BP avg (mb)'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='purple')
axes[1, 2].set_title('Barometric Pressure Distribution')
axes[1, 2].set_xlabel('Pressure (mb)')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()


# BIVARIATE ANALYSIS
print("\n4.2 BIVARIATE ANALYSIS - RELATIONSHIPS")

# Correlation matrix
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Matrix - Weather Variables', fontsize=16)
plt.tight_layout()
plt.show()

# Strong correlations
print("\nStrongest Correlations (|r| > 0.5):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.5:
            print(f"{correlation_matrix.columns[i]} ↔ {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}")

# Key scatter plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Key Relationships in Weather Data', fontsize=16)

# Temperature vs Humidity
axes[0, 0].scatter(df['2m T avg (F)'], df['RelHum avg 2m  (pct)'], alpha=0.3)
axes[0, 0].set_xlabel('Temperature (F)')
axes[0, 0].set_ylabel('Humidity (%)')
axes[0, 0].set_title('Temperature vs Humidity')

# Temperature vs Dew Point
axes[0, 1].scatter(df['2m T avg (F)'], df['2m DewPt avg (F)'], alpha=0.3, color='green')
axes[0, 1].set_xlabel('Temperature (F)')
axes[0, 1].set_ylabel('Dew Point (F)')
axes[0, 1].set_title('Temperature vs Dew Point')

# Solar Radiation vs Temperature
axes[1, 0].scatter(df['SolRad avg2m  (w/m^2)'], df['2m T avg (F)'], alpha=0.3, color='orange')
axes[1, 0].set_xlabel('Solar Radiation (w/m²)')
axes[1, 0].set_ylabel('Temperature (F)')
axes[1, 0].set_title('Solar Radiation vs Temperature')

# Wind Speed vs Temperature
axes[1, 1].scatter(df['10m Wind avg (mph)'], df['2m T avg (F)'], alpha=0.3, color='red')
axes[1, 1].set_xlabel('Wind Speed (mph)')
axes[1, 1].set_ylabel('Temperature (F)')
axes[1, 1].set_title('Wind Speed vs Temperature')

plt.tight_layout()
plt.show()

# CATEGORICAL ANALYSIS
print("\n4.3 CATEGORICAL ANALYSIS - BY STATION")

# Temperature by station
station_stats = df.groupby('FAWN Station').agg({
    '2m T avg (F)': ['mean', 'std', 'min', 'max'],
    '2m Rain tot (in)': 'sum',
    '10m Wind avg (mph)': 'mean',
    'RelHum avg 2m  (pct)': 'mean'
}).round(2)

print("\nStation-level Summary:")
print(station_stats.head(10))

# Box plot of temperature by station (top 10 stations)
top_stations = df['FAWN Station'].value_counts().head(10).index
df_top = df[df['FAWN Station'].isin(top_stations)]

plt.figure(figsize=(14, 6))
df_top.boxplot(column='2m T avg (F)', by='FAWN Station', figsize=(14, 6))
plt.title('Temperature Distribution by Station (Top 10)')
plt.suptitle('')
plt.xlabel('Station')
plt.ylabel('Temperature (F)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# TIME SERIES PATTERNS
print("\n4.4 TEMPORAL PATTERNS")

if 'Period' in df.columns:
    try:
        df['Period'] = pd.to_datetime(df['Period'])
        df_time = df.sort_values('Period')
        
        # Plot temperature over time
        plt.figure(figsize=(14, 6))
        plt.plot(df_time['Period'], df_time['2m T avg (F)'], alpha=0.5)
        plt.title('Temperature Over Time')
        plt.xlabel('Date')
        plt.ylabel('Temperature (F)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        print(" Temporal patterns visualized")
    except:
        print(" Could not analyze temporal patterns")

In [ ]:
print("STEP 5: TRANSFORMATION & FEATURE ENGINEERING")

print(f"\nOriginal shape: {df.shape}")

# DATETIME FEATURES
print("\n5.1 EXTRACT TEMPORAL FEATURES")

if 'Period' in df.columns:
    df['Period'] = pd.to_datetime(df['Period'], errors='coerce')
    df['Year'] = df['Period'].dt.year
    df['Month'] = df['Period'].dt.month
    df['Day'] = df['Period'].dt.day
    df['DayOfWeek'] = df['Period'].dt.dayofweek
    df['Quarter'] = df['Period'].dt.quarter
    df['WeekOfYear'] = df['Period'].dt.isocalendar().week
    df['Season'] = df['Month'].map({
        12: 'Winter', 1: 'Winter', 2: 'Winter',
        3: 'Spring', 4: 'Spring', 5: 'Spring',
        6: 'Summer', 7: 'Summer', 8: 'Summer',
        9: 'Fall', 10: 'Fall', 11: 'Fall'
    })
    print(" Created: Year, Month, Day, DayOfWeek, Quarter, WeekOfYear, Season")

# TEMPERATURE FEATURES
print("\n5.2 ENGINEER TEMPERATURE FEATURES")

# Temperature range (daily variation)
df['Temp_Range'] = df['2m T max (F)'] - df['2m T min (F)']

# Soil temperature range
df['Soil_Temp_Range'] = df['Tsoil max(avg)-10cm  (F)'] - df['Tsoil min(avg)-10cm  (F)']

# Temperature categories
df['Temp_Category'] = pd.cut(df['2m T avg (F)'], 
                              bins=[-np.inf, 32, 50, 70, 85, np.inf],
                              labels=['Freezing', 'Cold', 'Mild', 'Warm', 'Hot'])

# Heat index approximation (simplified)
df['Apparent_Temp'] = df['2m T avg (F)'] + 0.33 * df['RelHum avg 2m  (pct)'] / 100 * (df['2m T avg (F)'] - 70)

print(" Created: Temp_Range, Soil_Temp_Range, Temp_Category, Apparent_Temp")

# WIND FEATURES
print("\n5.3 ENGINEER WIND FEATURES")

# Wind speed range
df['Wind_Range'] = df['10m Wind max (mph)'] - df['10m Wind min (mph)']

# Wind categories
df['Wind_Category'] = pd.cut(df['10m Wind avg (mph)'],
                              bins=[0, 5, 10, 15, 25, np.inf],
                              labels=['Calm', 'Light', 'Moderate', 'Fresh', 'Strong'])

# Wind chill (when temp < 50F and wind > 3 mph)
mask = (df['2m T avg (F)'] < 50) & (df['10m Wind avg (mph)'] > 3)
df['Wind_Chill'] = np.where(
    mask,
    35.74 + 0.6215 * df['2m T avg (F)'] - 35.75 * (df['10m Wind avg (mph)'] ** 0.16) + 
    0.4275 * df['2m T avg (F)'] * (df['10m Wind avg (mph)'] ** 0.16),
    df['2m T avg (F)']
)

print(" Created: Wind_Range, Wind_Category, Wind_Chill")

# PRECIPITATION FEATURES
print("\n5.4 ENGINEER PRECIPITATION FEATURES")

# Rain intensity categories
df['Rain_Category'] = pd.cut(df['2m Rain tot (in)'],
                              bins=[-0.001, 0, 0.1, 0.5, 1.0, np.inf],
                              labels=['No Rain', 'Light', 'Moderate', 'Heavy', 'Very Heavy'])

# Rain flag
df['Has_Rain'] = (df['2m Rain tot (in)'] > 0).astype(int)

# Rain intensity (rain per 15min period)
df['Rain_Intensity'] = df['2m Rain max over 15min (in)'] / 0.25  # per hour

print(" Created: Rain_Category, Has_Rain, Rain_Intensity")

# HUMIDITY & DEW POINT FEATURES
print("\n5.5 ENGINEER HUMIDITY FEATURES")

# Humidity comfort categories
df['Humidity_Comfort'] = pd.cut(df['RelHum avg 2m  (pct)'],
                                 bins=[0, 30, 60, 80, 100],
                                 labels=['Dry', 'Comfortable', 'Humid', 'Very Humid'])

# Dew point comfort (based on dew point temperature)
df['Dewpoint_Comfort'] = pd.cut(df['2m DewPt avg (F)'],
                                bins=[-np.inf, 50, 60, 65, 70, np.inf],
                                labels=['Dry', 'Comfortable', 'Sticky', 'Uncomfortable', 'Oppressive'])

print(" Created: Humidity_Comfort, Dewpoint_Comfort")

# SOLAR RADIATION FEATURES
print("\n5.6 ENGINEER SOLAR FEATURES")

# Solar radiation categories
df['Solar_Category'] = pd.cut(df['SolRad avg2m  (w/m^2)'],
                               bins=[0, 200, 400, 600, 800, np.inf],
                               labels=['Low', 'Moderate', 'High', 'Very High', 'Extreme'])

print(" Created: Solar_Category")

# COMPOSITE FEATURES
print("\n5.7 CREATE COMPOSITE INDICES")

# Comfort index (normalized combination of temp, humidity, wind)
scaler = StandardScaler()
comfort_features = df[['2m T avg (F)', 'RelHum avg 2m  (pct)', '10m Wind avg (mph)']].copy()
comfort_scaled = scaler.fit_transform(comfort_features.fillna(comfort_features.mean()))
df['Comfort_Index'] = 100 - np.mean(np.abs(comfort_scaled), axis=1) * 20

# Weather severity index (combination of extremes)
df['Weather_Severity'] = (
    np.abs(df['2m T avg (F)'] - 70) / 20 +  # Temperature deviation
    df['10m Wind avg (mph)'] / 25 +  # Wind factor
    df['2m Rain tot (in)'] * 2  # Rain factor
)

print(" Created: Comfort_Index, Weather_Severity")

# STATION AGGREGATIONS
print("\n5.8 CREATE STATION-LEVEL FEATURES")

# Average conditions by station
station_avg_temp = df.groupby('FAWN Station')['2m T avg (F)'].transform('mean')
df['Station_Avg_Temp'] = station_avg_temp

station_avg_rain = df.groupby('FAWN Station')['2m Rain tot (in)'].transform('mean')
df['Station_Avg_Rain'] = station_avg_rain

# Deviation from station average
df['Temp_Deviation_From_Station'] = df['2m T avg (F)'] - df['Station_Avg_Temp']

print(" Created: Station_Avg_Temp, Station_Avg_Rain, Temp_Deviation_From_Station")

# INTERACTION FEATURES
print("\n5.9 CREATE INTERACTION FEATURES")

# Temperature * Humidity interaction
df['Temp_Humidity_Interaction'] = df['2m T avg (F)'] * df['RelHum avg 2m  (pct)']

# Wind * Rain interaction (storm indicator)
df['Wind_Rain_Interaction'] = df['10m Wind avg (mph)'] * df['2m Rain tot (in)']

print(" Created: Temp_Humidity_Interaction, Wind_Rain_Interaction")

# SAVE TRANSFORMED DATA
print("\n5.10 SAVE FEATURE-ENGINEERED DATASET")
df.to_csv('FAWN_report_features.csv', index=False)
print(f" Saved: FAWN_report_features.csv")
print(f"Final shape: {df.shape}")
print(f"New features created: {df.shape[1] - 20}")

# VISUALIZE NEW FEATURES
print("\n5.11 VISUALIZE ENGINEERED FEATURES")

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Engineered Features Visualizations', fontsize=16)

# Temperature by Season
df.boxplot(column='2m T avg (F)', by='Season', ax=axes[0, 0])
axes[0, 0].set_title('Temperature by Season')
axes[0, 0].set_xlabel('Season')
axes[0, 0].set_ylabel('Temperature (F)')

# Rain by Category
df['Rain_Category'].value_counts().plot(kind='bar', ax=axes[0, 1], color='blue')
axes[0, 1].set_title('Rain Category Distribution')
axes[0, 1].set_xlabel('Category')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)

# Comfort Index distribution
axes[0, 2].hist(df['Comfort_Index'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[0, 2].set_title('Comfort Index Distribution')
axes[0, 2].set_xlabel('Comfort Index')
axes[0, 2].set_ylabel('Frequency')

# Weather Severity
axes[1, 0].hist(df['Weather_Severity'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='red')
axes[1, 0].set_title('Weather Severity Index')
axes[1, 0].set_xlabel('Severity')
axes[1, 0].set_ylabel('Frequency')

# Temperature Range
axes[1, 1].hist(df['Temp_Range'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 1].set_title('Daily Temperature Range')
axes[1, 1].set_xlabel('Range (F)')
axes[1, 1].set_ylabel('Frequency')

# Wind Category
df['Wind_Category'].value_counts().plot(kind='bar', ax=axes[1, 2], color='purple')
axes[1, 2].set_title('Wind Category Distribution')
axes[1, 2].set_xlabel('Category')
axes[1, 2].set_ylabel('Count')
axes[1, 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
print("STEP 6: SAVE & DOCUMENT")

# CREATE DATA DICTIONARY
print("\n6.1 GENERATE DATA DICTIONARY")

data_dict = {
    'Column_Name': [],
    'Data_Type': [],
    'Description': [],
    'Non_Null_Count': [],
    'Unique_Values': [],
    'Sample_Values': []
}

for col in df.columns:
    data_dict['Column_Name'].append(col)
    data_dict['Data_Type'].append(str(df[col].dtype))
    data_dict['Non_Null_Count'].append(df[col].notna().sum())
    data_dict['Unique_Values'].append(df[col].nunique())
    
    # Sample values
    if df[col].dtype in ['object', 'category']:
        sample = df[col].dropna().unique()[:3].tolist()
    else:
        sample = [f"{df[col].min():.2f}", f"{df[col].mean():.2f}", f"{df[col].max():.2f}"]
    data_dict['Sample_Values'].append(str(sample))
    
    # Descriptions
    descriptions = {
        'FAWN Station': 'Weather station identifier',
        'Period': 'Date/time of observation',
        '2m T avg (F)': 'Average air temperature at 2m height',
        '2m T min (F)': 'Minimum air temperature at 2m height',
        '2m T max (F)': 'Maximum air temperature at 2m height',
        'Temp_Range': 'Daily temperature range (max - min)',
        'Temp_Category': 'Categorized temperature (Freezing/Cold/Mild/Warm/Hot)',
        'Season': 'Season of the year',
        'Rain_Category': 'Rainfall intensity category',
        'Comfort_Index': 'Weather comfort score (0-100)',
        'Weather_Severity': 'Weather severity index',
    }
    data_dict['Description'].append(descriptions.get(col, 'See documentation'))

dict_df = pd.DataFrame(data_dict)
dict_df.to_csv('FAWN_data_dictionary.csv', index=False)
print("✓ Data dictionary saved: FAWN_data_dictionary.csv")
print(dict_df.head(10))

# GENERATE ANALYSIS REPORT
print("\n6.2 CREATE ANALYSIS SUMMARY REPORT")

# Calculate statistics safely
avg_temp = df['2m T avg (F)'].mean()
temp_min = df['2m T min (F)'].min()
temp_max = df['2m T max (F)'].max()
total_rain = df['2m Rain tot (in)'].sum()
rain_days = (df['2m Rain tot (in)'] > 0).sum()
rain_pct = (rain_days / len(df) * 100) if len(df) > 0 else 0
avg_rain_per_event = df[df['2m Rain tot (in)'] > 0]['2m Rain tot (in)'].mean() if rain_days > 0 else 0
avg_wind = df['10m Wind avg (mph)'].mean()
max_wind = df['10m Wind max (mph)'].max()
avg_humidity = df['RelHum avg 2m  (pct)'].mean()
num_stations = df['FAWN Station'].nunique()
most_active = df['FAWN Station'].value_counts().index[0]
records_per_station = len(df) / num_stations if num_stations > 0 else 0

# Correlations
temp_dewpt_corr = df[['2m T avg (F)', '2m DewPt avg (F)']].corr().iloc[0,1]
temp_soil_corr = df[['2m T avg (F)', 'Tsoil avg-10cm  (F)']].corr().iloc[0,1]
temp_humid_corr = df[['2m T avg (F)', 'RelHum avg 2m  (pct)']].corr().iloc[0,1]

# Optional features
temp_category = df['Temp_Category'].mode()[0] if 'Temp_Category' in df.columns and len(df['Temp_Category'].mode()) > 0 else 'N/A'
wind_category = df['Wind_Category'].mode()[0] if 'Wind_Category' in df.columns and len(df['Wind_Category'].mode()) > 0 else 'N/A'
avg_comfort = df['Comfort_Index'].mean() if 'Comfort_Index' in df.columns else 'N/A'

report = f"""
FAWN WEATHER DATA - EXPLORATORY DATA ANALYSIS REPORT
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

DATASET OVERVIEW
Original Records: 6,572
Final Records: {len(df):,}
Original Features: 20
Engineered Features: {len(df.columns) - 20}
Total Features: {len(df.columns)}

DATA QUALITY SUMMARY
Missing Values Handled: Yes
Duplicates Removed: Yes
Outliers Flagged: Yes
Consistency Issues Fixed: Yes
Invalid Records Removed: Yes

KEY FINDINGS

1. TEMPERATURE PATTERNS
   - Average Temperature: {avg_temp:.2f}°F
   - Temperature Range: {temp_min:.2f}°F to {temp_max:.2f}°F
   - Most Common Category: {temp_category}

2. PRECIPITATION PATTERNS
   - Total Rainfall Recorded: {total_rain:.2f} inches
   - Days with Rain: {rain_days} ({rain_pct:.1f}%)
   - Average Rain per Event: {avg_rain_per_event:.3f} inches

3. WIND PATTERNS
   - Average Wind Speed: {avg_wind:.2f} mph
   - Max Wind Speed: {max_wind:.2f} mph
   - Predominant Wind Category: {wind_category}

4. HUMIDITY & COMFORT
   - Average Humidity: {avg_humidity:.1f}%
   - Average Comfort Index: {avg_comfort if isinstance(avg_comfort, str) else f'{avg_comfort:.2f}'}
   
5. STATION ANALYSIS
   - Number of Stations: {num_stations}
   - Most Active Station: {most_active}
   - Records per Station (avg): {records_per_station:.0f}

CORRELATIONS DISCOVERED
Strong positive correlations:
- Temperature <-> Dew Point: {temp_dewpt_corr:.3f}
- Air Temp <-> Soil Temp: {temp_soil_corr:.3f}

Strong negative correlations:
- Temperature <-> Humidity: {temp_humid_corr:.3f}

FEATURE ENGINEERING SUMMARY
Created Features:
+ Temporal: Year, Month, Season, Quarter, DayOfWeek
+ Temperature: Temp_Range, Temp_Category, Apparent_Temp, Wind_Chill
+ Wind: Wind_Range, Wind_Category, Wind_Chill
+ Precipitation: Rain_Category, Has_Rain, Rain_Intensity
+ Composite: Comfort_Index, Weather_Severity
+ Station-level: Station averages and deviations
+ Interactions: Temp*Humidity, Wind*Rain

DATA QUALITY ACTIONS TAKEN
1. Removed {6572 - len(df)} rows with data quality issues
2. Fixed min/max inversions in temperature, wind, and soil temp
3. Interpolated missing values where appropriate
4. Flagged extreme outliers for investigation
5. Validated all measurements within reasonable ranges
"""

# Save report
with open('FAWN_analysis_report.txt', 'w', encoding='utf-8') as f:
    f.write(report)

print("✓ Analysis report saved: FAWN_analysis_report.txt")
print(report)

# === CREATE FINAL VISUALIZATIONS ===
print("\n6.3 GENERATE FINAL SUMMARY VISUALIZATIONS")
print("-" * 50)

fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Title
fig.suptitle('FAWN Weather Data - Executive Summary Dashboard', fontsize=18, fontweight='bold')

# 1. Temperature Over Time
if 'Period' in df.columns:
    ax1 = fig.add_subplot(gs[0, :])
    df_sorted = df.sort_values('Period')
    ax1.plot(pd.to_datetime(df_sorted['Period']), df_sorted['2m T avg (F)'], alpha=0.6)
    ax1.set_title('Temperature Trend Over Time', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Temperature (°F)')
    ax1.grid(True, alpha=0.3)

# 2. Temperature Distribution by Season
ax2 = fig.add_subplot(gs[1, 0])
if 'Season' in df.columns:
    df.boxplot(column='2m T avg (F)', by='Season', ax=ax2)
    ax2.set_title('Temperature by Season', fontsize=10, fontweight='bold')
    ax2.set_xlabel('Season')
    ax2.set_ylabel('Temperature (°F)')
    plt.sca(ax2)
    plt.xticks(rotation=45)

# 3. Rainfall Distribution
ax3 = fig.add_subplot(gs[1, 1])
rain_data = df[df['2m Rain tot (in)'] > 0]['2m Rain tot (in)']
ax3.hist(rain_data, bins=30, edgecolor='black', alpha=0.7, color='blue')
ax3.set_title('Rainfall Distribution (Rain Days Only)', fontsize=10, fontweight='bold')
ax3.set_xlabel('Rainfall (inches)')
ax3.set_ylabel('Frequency')

# 4. Wind Speed Distribution
ax4 = fig.add_subplot(gs[1, 2])
ax4.hist(df['10m Wind avg (mph)'].dropna(), bins=30, edgecolor='black', alpha=0.7, color='orange')
ax4.set_title('Wind Speed Distribution', fontsize=10, fontweight='bold')
ax4.set_xlabel('Wind Speed (mph)')
ax4.set_ylabel('Frequency')

# 5. Temperature vs Humidity
ax5 = fig.add_subplot(gs[2, 0])
ax5.scatter(df['2m T avg (F)'], df['RelHum avg 2m  (pct)'], alpha=0.3, s=10)
ax5.set_title('Temperature vs Humidity', fontsize=10, fontweight='bold')
ax5.set_xlabel('Temperature (°F)')
ax5.set_ylabel('Humidity (%)')
ax5.grid(True, alpha=0.3)

# 6. Station Activity
ax6 = fig.add_subplot(gs[2, 1])
station_counts = df['FAWN Station'].value_counts().head(10)
station_counts.plot(kind='barh', ax=ax6, color='green')
ax6.set_title('Top 10 Stations by Records', fontsize=10, fontweight='bold')
ax6.set_xlabel('Number of Records')
ax6.set_ylabel('Station')

# 7. Weather Severity
ax7 = fig.add_subplot(gs[2, 2])
if 'Weather_Severity' in df.columns:
    ax7.hist(df['Weather_Severity'].dropna(), bins=30, edgecolor='black', alpha=0.7, color='red')
    ax7.set_title('Weather Severity Index', fontsize=10, fontweight='bold')
    ax7.set_xlabel('Severity Score')
    ax7.set_ylabel('Frequency')

plt.savefig('FAWN_summary_dashboard.png', dpi=300, bbox_inches='tight')
print("✓ Summary dashboard saved: FAWN_summary_dashboard.png")
plt.show()